<a href="https://colab.research.google.com/github/5791nbm/FinCoLab/blob/main/07%20VaR%20Value%20at%20Risk.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Value at risk

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import datetime as dt
from scipy.stats import norm

In [ ]:
def download_data(stock, start_date, end_date):
  data = {}
  ticker = yf.download(stock, start_date, end_date, auto_adjust= False)
  #This is a line of code I added to handle changes with yfinance library
  ticker = ticker.xs(stock, axis= 1, level= 'Ticker')
  data[stock] = ticker['Adj Close']
  #ticker = ticker.rename(columns= {'Adj Close': stock})
  return pd.DataFrame(data)

#To calculate the value at risk tomorrow (n=1)
def calc_VaR(position, c, mu, signma):
  var = position * (mu - signma * norm.ppf(1-c))
  return var

#To calculate the value at risk tfor any days in the future (n=1)
def calc_VaR_n(position, c, mu, signma, n):
  var = position * (mu*n - signma * np.sqrt(n) * norm.ppf(1-c))
  return var

if __name__ == '__main__':

  start = dt.datetime(2014, 1, 1)
  end = dt.datetime(2018, 1, 1)

  stock_data = download_data('C', start, end)
  #print(stock_data)

  stock_data['returns'] = np.log(stock_data['C'] / stock_data['C'].shift(1))
  stock_data = stock_data.dropna()

  #Define investment scenario
  S= 1e6 #investing 1 million dollars
  c= 0.95 #confidence level of 95%

  #Assume that daily returns are normally distibuted
  mu = np.mean(stock_data['returns'])
  signma = np.std(stock_data['returns'])
  print()
  print('The one day VaR (Value at Risk) is: $%0.2f' % calc_VaR(S, c, mu, signma))
  print('The ten days VaR (Value at Risk) is: $%0.2f' % calc_VaR_n(S, c, mu, signma, 10))
  print()
  print(stock_data)


[*********************100%***********************]  1 of 1 completed


The one day VaR (Value at Risk) is: $25396.71
The ten days VaR (Value at Risk) is: $82897.97

                    C   returns
Date                           
2014-01-03  40.842766  0.021388
2014-01-06  41.156345  0.007648
2014-01-07  41.439346  0.006853
2014-01-08  41.921196  0.011561
2014-01-09  42.219498  0.007091
...               ...       ...
2017-12-22  59.340660 -0.004362
2017-12-26  58.782551 -0.009450
2017-12-27  58.869026  0.001470
2017-12-28  59.018379  0.002534
2017-12-29  58.491714 -0.008964

[1006 rows x 2 columns]


VaR Monte Carlo

In [41]:
def download_data(stock, start_date, end_date):
  data = {}
  ticker = yf.download(stock, start_date, end_date, auto_adjust= False)
  #This is a line of code I added to handle changes with yfinance library
  ticker = ticker.xs(stock, axis= 1, level= 'Ticker')
  data[stock] = ticker['Adj Close']
  #ticker = ticker.rename(columns= {'Adj Close': stock})
  return pd.DataFrame(data)

class VaR_Monte_Carlo:
  def __init__(self, S, mu, sigma, c, n, iterations):
    # this is the value of the investment at t = 0
    self.S = S
    self.mu = mu
    self.sigma = sigma
    self.c = c
    self.n = n
    self.iterations = iterations

  def simulation(self):
    rand = np.random.normal(0, 1, [1, self.iterations])
    stock_price = self.S * np.exp(self.n * (self.mu - 0.5 * self.sigma ** 2) + self.sigma * np.sqrt(self.n) * rand)
    #sort the stock prices, confidence level: 95% -> 5, 99% -> 1
    stock_price = np.sort(stock_price)
    percentile = np.percentile(stock_price, (1-self.c)*100)
    return self.S - percentile

if __name__ == '__main__':

  S= 1e6
  c= 0.99
  n= 1
  iterations = 100000

  #Historical data
  start = dt.datetime(2014, 1, 1)
  end = dt.datetime(2017, 10, 15)

  #Stock data
  citi = download_data('C', start, end)
  #citi['returns'] = citi['Adjust Close'].pct_change()
  citi['returns'] = np.log(citi['C'] / citi['C'].shift(1))
  citi = citi.dropna()

  #get mean and std
  mu = np.mean(citi['returns'])
  sigma = np.std(citi['returns'])

  model = VaR_Monte_Carlo(S, mu, sigma, c, n, iterations)
  print()
  print('VaR with Monte Carlo simulation: $%.2f' % model.simulation())

[*********************100%***********************]  1 of 1 completed


VaR with Monte Carlo simulation: $35217.12
